# Targeting model test

James Yu, 29 June 2022

In [1]:
import numpy as np
import scipy.optimize

Take the following 2x2 matrix and opinions:

In [2]:
A = np.array([
    [0.6, 0.4],
    [0.4, 0.6]
])
x_0 = np.array([[-5.0, 10.0]], ndmin = 2).T
delta = 0.9
I = np.identity(2)

We'll need the eigendecomposition:

In [3]:
eigvals, U = np.linalg.eig(A)
D = np.diag(eigvals)
np.allclose(U @ D @ U.T, A)

True

To compute $\mu$, we need a $C$ that is less than the cutoff for where $\mu = 0$ is optimal:

In [4]:
C = (x_0.T @ x_0).item() // 2 # ensure nonzero mu or else tilde_z = tilde_x_0 is optimal
C

62.0

To find $\mu$ given $C$, we take the budget constraint at equality and find the $\mu$ which satisfies this constraint. This uses the constraint from the note:

In [5]:
def objective_function(mu, delta, D, x_0, U, C):
    I = np.identity(2)
    # np.linalg.inv is the matrix inverse function
    tilde_z = -np.linalg.inv(np.linalg.inv(I - delta * D @ D) + mu * I) @ np.linalg.inv(I - delta * D @ D) @ U.T @ x_0
    return C - (tilde_z.T @ tilde_z).item() # .item() converts from a 1x1 vector type to a 1x1 scalar type for compatibility

So if we plug in e.g. $\mu = 2$, the disparity between $C$ and the budget constraint is:

In [6]:
objective_function(2, delta, D, x_0, U, C)

40.19713189405476

This variant of the function is the FOC with respect to $z$ only, replacing all "tilde" variables with their original counterparts as e.g. $U^\prime z$ instead of $\tilde z$:

In [7]:
def objective_function_2(mu, delta, D, x_0, U, C):
    I = np.identity(2)
    z = -np.linalg.inv(U @ np.linalg.inv(I - delta * D @ D) @ U.T + mu * I) @ U @ np.linalg.inv(I - delta * D @ D) @ U.T @ x_0
    return C - (z.T @ z).item()

In [8]:
objective_function_2(2, delta, D, x_0, U, C)

40.19713189405478

The disparity produced by $\mu = 2$ is the exact same, since the original budget constraint is $\tilde z^\prime \tilde z = C = (U^\prime z)^\prime U^\prime z = z^\prime U U^\prime z = z^\prime I z = z^\prime z$. This means the $\mu$ should be the exact same (ignoring rounding error) for each FOC:

In [9]:
mu = scipy.optimize.newton(lambda mu: objective_function(mu, delta, D, x_0, U, C), 1e-100)
mu

0.5082007315904419

In [10]:
mu_2 = scipy.optimize.newton(lambda mu: objective_function_2(mu, delta, D, x_0, U, C), 1e-100)
mu_2

0.5082007315904422

In [11]:
objective_function(mu, delta, D, x_0, U, C)

0.0

In [12]:
objective_function_2(mu_2, delta, D, x_0, U, C)

-7.105427357601002e-15

Now, if we take the $\tilde z$ FOC and substitute the optimal $\mu$, we get the following result for $\tilde z$:

In [13]:
tilde_z = -np.linalg.inv(np.linalg.inv(I - delta * D @ D) + mu * I) @ np.linalg.inv(I - delta * D @ D) @ U.T @ x_0
tilde_z

array([[-3.36454736],
       [-7.11897612]])

If instead we substitute it into the $z$ FOC, we get a result for $z$ that is wrong because the element $z^2$ should be zero:

In [14]:
z = -np.linalg.inv(U @ np.linalg.inv(I - delta * D @ D) @ U.T + mu * I) @ U @ np.linalg.inv(I - delta * D @ D) @ U.T @ x_0
z

array([[ 2.65478204],
       [-7.41297055]])

But notice that when we compute $\tilde z$ from this result, we get:

In [15]:
U.T @ z

array([[-3.36454736],
       [-7.11897612]])

And likewise, if we take the computationally-generated $\tilde z$ and attempt to back out $z$ from it using the fact that $\tilde z = U^\prime z$ implies $U \tilde z = U U^\prime z = I z = z$, we get:

In [16]:
U @ tilde_z

array([[ 2.65478204],
       [-7.41297055]])

which is the exact same $z$ as was computationally generated from the $z$ FOC, which is again incorrect since the second element is supposed to be zero.